In [14]:
!pip install -q pytorch-lightning==1.6.4 neptune-client transformers sentencepiece

^C


In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch

from transformers import AutoTokenizer

import pytorch_lightning as pl

import pickle

RANDOM_SEED = 42
COLAB = True

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [ ]:
torch.cuda.is_available()

In [ ]:
if COLAB:
    import os
    os.getcwd()
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
cd ./drive/MyDrive/human_value/human_values_behind_arguments

In [ ]:
!git pull

In [20]:
from data_modules.BertDataModule import BertDataset
from models.BertFineTunerPl import BertFineTunerPl

In [2]:
PARAMS_ENSEMBLE = {
    "MODEL_CHECKPOINTS": ['./checkpoints/HCV-409-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-408-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-406-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-402-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-403-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-405-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-364-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-366-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-368-microsoft-deberta-large-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-371-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-372-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt',
                          './checkpoints/HCV-375-danschr-roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165-BS_8-LR_2e-05-HL_None-DROPOUT_None-SL_None.ckpt'
                          ],
    "DESCRIPTION":"FULL #3xDebL_F1 3EP 3xdanRobL_F1 3EP 3xDebL_Loss 3EP 3xdanRobL_Loss 3EP",
    "TEST_PATH" : "./data/arguments-test.tsv",
    "MAX_THRESHOLD_METRIC": "custom",
    "ENSEMBLE": "EN",
    "ENSEMBLE_THRESHOLD":0.26,
    "LABEL_COLUMNS":['Self-direction: thought',
                     'Self-direction: action',
                     'Stimulation',
                     'Hedonism',
                     'Achievement',
                     'Power: dominance',
                     'Power: resources',
                     'Face',
                     'Security: personal',
                     'Security: societal',
                     'Tradition',
                     'Conformity: rules',
                     'Conformity: interpersonal',
                     'Humility',
                     'Benevolence: caring',
                     'Benevolence: dependability',
                     'Universalism: concern',
                     'Universalism: nature',
                     'Universalism: tolerance',
                     'Universalism: objectivity']
}

In [3]:
NAME = ""
ids = []
for elem in PARAMS_ENSEMBLE["MODEL_CHECKPOINTS"]:
    text_list = elem.split("checkpoints/")[1]
    text_list = text_list.split("-")
    id = text_list[0]+"-" + text_list[1]
    ids.append(id)
    NAME= NAME + "_" + id
    print(text_list[0]+"-" + text_list[1])
NAME = PARAMS_ENSEMBLE["ENSEMBLE"]+"_"+NAME[1:]

PARAMS_ENSEMBLE["IDS"] = ids
LABEL_COLUMNS = PARAMS_ENSEMBLE["LABEL_COLUMNS"]

HCV-409
HCV-408
HCV-406
HCV-402
HCV-403
HCV-405
HCV-364
HCV-366
HCV-368
HCV-371
HCV-372
HCV-375


# Load Data

# Defining the Model PARAMS

In [ ]:
#Loading the parameters for each model
PARAMS_LIST = []
for id in PARAMS_ENSEMBLE["IDS"]:
    with open(f'./checkpoints/{id}_PARAMS.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
        PARAMS_LIST.append(loaded_dict)

In [36]:
# Concatenating relevant information into one Ensemble_list
ENSEMBLE_LIST = []
for param, id, mc in zip(PARAMS_LIST, PARAMS_ENSEMBLE["IDS"], PARAMS_ENSEMBLE["MODEL_CHECKPOINTS"]):
    ENSEMBLE_LIST.append({"PARAMS":param, "ID":id,"MODEL_CHECKPOINT":mc})

[{'PARAMS': {'MODEL_PATH': 'danschr/roberta-large-BS_16-EPOCHS_8-LR_5e-05-ACC_GRAD_2-MAX_LENGTH_165',
   'BATCH_SIZE': 8,
   'LR': 2e-05,
   'EPOCHS': 20,
   'DEVICE': device(type='cuda'),
   'NUM_TRAIN_WORKERS': 4,
   'NUM_VAL_WORKERS': 4,
   'MAX_TOKEN_COUNT': 165,
   'SEED': 42,
   'WEIGHTS': [0.54238067,
    0.38692124,
    2.28156376,
    3.17629465,
    0.36239603,
    0.89006059,
    0.87405231,
    1.44634846,
    0.27533886,
    0.31700788,
    0.97389395,
    0.46999331,
    2.6127585,
    1.34618582,
    0.41359411,
    0.70227324,
    0.26240447,
    1.32057903,
    0.82508163,
    0.52087147],
   'WEIGHTS_APPLIED': False,
   'CRITERION': [BCEWithLogitsLoss()],
   'DATA': 'Premise + Stance + Conclusion',
   'OPTIMIZER': 'AdamW',
   'ACCUMULATE_GRAD_BATCHES': 1,
   'PATIENCE': 3,
   'VAL_CHECK_INTERVAL': 300,
   'MAX_THRESHOLD_METRIC': 'custom',
   'EARLY_STOPPING_METRIC': 'avg_val_loss',
   'EARLY_STOPPING_MODE': 'min',
   'ACTIVATION': None,
   'DROPOUT': None,
   'HIDDEN_

# Predict Test File

In [ ]:
test_df_input = pd.read_csv(PARAMS_ENSEMBLE["TEST_PATH"], sep='\t')

In [ ]:
test_df_input["text"] = test_df_input["Premise"]+" " + test_df_input["Stance"]+ " " + test_df_input["Conclusion"]

In [ ]:
def predict_unseen_data(trained_model, data):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    trained_model = trained_model.to(device)

    silver_df_dataset = BertDataset(
        data=data,
        tokenizer=TOKENIZER,
        max_token_count=PARAMS["MAX_TOKEN_COUNT"],
    )

    predictions = []

    for item in tqdm(silver_df_dataset):
        _, prediction = trained_model(
            item["input_ids"].unsqueeze(dim=0).to(device),
            item["attention_mask"].unsqueeze(dim=0).to(device)
        )
        predictions.append(prediction.flatten())

    predictions = torch.stack(predictions).detach().cpu()

    return predictions

In [ ]:
predictions = []
for idx, elem in enumerate(ENSEMBLE_LIST):
    print(f"Starting with model {elem['MODEL_CHECKPOINT']}")
    PARAMS = elem["PARAMS"]
    trained_model = BertFineTunerPl.load_from_checkpoint(
        elem["MODEL_CHECKPOINT"],
        params=PARAMS,
        label_columns=LABEL_COLUMNS,
        n_classes=len(LABEL_COLUMNS)
    )
    trained_model.eval()
    trained_model.freeze()
    print(f"With Tokenizer {PARAMS['MODEL_PATH']}")
    TOKENIZER = AutoTokenizer.from_pretrained(PARAMS["MODEL_PATH"])
    pred = predict_unseen_data(trained_model=trained_model, data=test_df_input)
    predictions.append(pred)

In [ ]:
predictions = torch.stack(predictions).numpy()
predictions_avg = np.mean(predictions, axis=0)

In [ ]:
upper, lower = 1, 0

# Use optimal decision threshold.
y_pred = np.where(predictions_avg > PARAMS_ENSEMBLE["ENSEMBLE_THRESHOLD"], upper, lower)

In [ ]:
prediction_dictionary = {}
prediction_dictionary["Argument ID"] = test_df_input["Argument ID"]
for idx, l_name in enumerate(LABEL_COLUMNS):
    prediction_dictionary[l_name]=y_pred[:,idx]

test_prediction_df = pd.DataFrame(prediction_dictionary)
test_prediction_df.head()

In [4]:
test_prediction_df.to_csv(f"./submission_test.tsv", sep="\t", index=False)

NameError: name 'test_prediction_df' is not defined